In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
import pickle

In [29]:
(_, _), (x_test, y_test) = cifar10.load_data()

In [31]:
with open('saved_images_with_labels.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# Восстанавливаем тензоры из numpy
x_train = [tf.convert_to_tensor(img) for img in loaded_data['images']]
y_train = loaded_data['labels']  # список лейблов
y_train = tf.keras.utils.to_categorical(y_train, 10)

x_test = x_test / 255.0
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [33]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Перемешиваем с фиксированным seed (чтобы результат был воспроизводим)
dataset = dataset.shuffle(buffer_size=len(x_train), seed=42)

In [34]:
for i, (image, label) in enumerate(dataset.take(10)):
    print(f"Элемент {i}: Лейбл = {label}, Размер картинки = {image.shape}")

Элемент 0: Лейбл = [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 1: Лейбл = [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 2: Лейбл = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 3: Лейбл = [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 4: Лейбл = [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 5: Лейбл = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 6: Лейбл = [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 7: Лейбл = [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 8: Лейбл = [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)
Элемент 9: Лейбл = [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], Размер картинки = (1, 32, 32, 3)


In [35]:
print("x_test.shape:", x_test.shape)  # должно быть (n, ...)
print("y_test.shape:", y_test.shape)

x_test.shape: (10000, 32, 32, 3)
y_test.shape: (10000, 10)


In [32]:
print(y_test[0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [36]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

In [37]:
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

In [38]:
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

In [39]:
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

In [40]:
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [41]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.save_weights('initial_weights.weights.h5')
history = model.fit(dataset, epochs=8,validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/8


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(10,), output.shape=(1, 10)

x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 10)
Sample y_test value: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [44]:
with open('saved_images_with_labels.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# Восстанавливаем тензоры из numpy
x_train = tf.convert_to_tensor([img for img in loaded_data['images']])  # Исправлено
y_train = loaded_data['labels']  # список лейблов
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_train = tf.convert_to_tensor(y_train)  # Преобразуем в тензор

x_test = x_test / 255.0
y_test = tf.keras.utils.to_categorical(y_test, 10)

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))  # Исправлено
dataset = dataset.shuffle(buffer_size=len(x_train), seed=42)


In [45]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (5000, 10)
y_test shape: (10000, 10, 10)
